# VITS

## Download

### Download Model VITS

In [1]:
# Download model, and save
# https://huggingface.co/kakao-enterprise/vits-ljs
from transformers import VitsModel, AutoTokenizer
import torch

models_folder_path = "./../download/model"
model_name = "kakao-enterprise/vits-ljs"
model = VitsModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

model.save_pretrained(models_folder_path+ "/vits")
tokenizer.save_pretrained(models_folder_path+ "/vits")

2025-05-30 17:23:49.021090: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748618629.167147   19997 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748618629.209004   19997 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748618629.510456   19997 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1748618629.510498   19997 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1748618629.510500   19997 computation_placer.cc:177] computation placer alr

('./../download/model/vits/tokenizer_config.json',
 './../download/model/vits/special_tokens_map.json',
 './../download/model/vits/vocab.json',
 './../download/model/vits/added_tokens.json')

## RUN

In [8]:
from transformers import VitsModel, AutoTokenizer
from IPython.display import Audio
import time

# model = VitsModel.from_pretrained("kakao-enterprise/vits-ljs")
# tokenizer = AutoTokenizer.from_pretrained("kakao-enterprise/vits-ljs")
model = VitsModel.from_pretrained(
            pretrained_model_name_or_path="./../download/model/vits",
            local_files_only=True
        ).to("cpu")
tokenizer =  AutoTokenizer.from_pretrained(
            pretrained_model_name_or_path="./../download/model/vits", 
            local_files_only=True
        )

start = time.time()
text= "specifies the required processes implemented in the engineering activities."

print(time.time() - start)
inputs = tokenizer(text, return_tensors="pt")

print(time.time() - start)
with torch.no_grad():
    output = model(**inputs).waveform

print(time.time() - start)

Audio(output, rate=model.config.sampling_rate)


5.269050598144531e-05
0.6236364841461182
3.679344415664673


# SpeechT5

## Download

### Download Model SpeechT5

In [9]:
from transformers import VitsModel, AutoTokenizer
import torch
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech


models_folder_path = "./../download/model"
processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")

processor.save_pretrained(models_folder_path+ "/speecht5/processor")
model.save_pretrained(models_folder_path+ "/speecht5/model")

/mnt/c/Users/Dawid/Desktop/f19/fastapi1/.venv/lib/python3.10/site-packages/transformers/modeling_utils.py:3353: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 1876}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


### Download dataset

In [10]:
from datasets import load_dataset, load_from_disk

embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")

datasets_folder_path = "./../download/dataset/embeddings_dataset"
embeddings_dataset.save_to_disk(datasets_folder_path)

Saving the dataset (0/1 shards):   0%|          | 0/7931 [00:00<?, ? examples/s]

### Download vocoder

In [11]:
from transformers import SpeechT5HifiGan

models_folder_path = "./../download/vocoder"
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")

vocoder.save_pretrained(models_folder_path+ "/hifigan/")

## Run

In [14]:
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech

# processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
# model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")

processor = SpeechT5Processor.from_pretrained(
            pretrained_model_name_or_path="./../download/model/speecht5/processor",
            local_files_only=True
        )
model =  SpeechT5ForTextToSpeech.from_pretrained(
            pretrained_model_name_or_path="./../download/model/speecht5/model",
            local_files_only=True
        );

In [15]:
inputs = processor(text="Don't count the days, make the days count.", return_tensors="pt")

In [16]:
from datasets import load_dataset

embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")

import torch
speaker_embeddings = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)

In [17]:
spectrogram = model.generate_speech(inputs["input_ids"], speaker_embeddings)

In [6]:
from transformers import SpeechT5HifiGan

# vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")
vocoder = SpeechT5HifiGan.from_pretrained(pretrained_model_name_or_path="./../download/vocoder/speecht5/processor", local_files_only=True)

config.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/50.7M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/50.6M [00:00<?, ?B/s]

In [7]:
speech = model.generate_speech(inputs["input_ids"], speaker_embeddings, vocoder=vocoder)

In [8]:
from IPython.display import Audio

Audio(speech, rate=16000)